## Bike Rental Data Management Project

## Library and Data Imports

In [2]:
import glob
import pandas as pd

## Citi Bike Data Overview

The Citi Bike data is stored in monthly files. Let's read those in and concatenate them in a single dataframe.

In [3]:
files = glob.glob("../data/raw/JC-2016*-citibike-tripdata.csv")
df_list = []

for filename in files:
  data = pd.read_csv(filename)
  df_list.append(data)
df = pd.concat(df_list)

Let's visually inspect the beginning and end of the dataset

In [12]:
df.head(2)

,Trip Duration,Start Time,Stop Time,Start Station ID,Start Station Name,Start Station Latitude,Start Station Longitude,End Station ID,End Station Name,End Station Latitude,End Station Longitude,Bike ID,User Type,Birth Year,Gender
0,204,2016-12-01 00:33:31,2016-12-01 00:36:55,3186,Grove St PATH,40.719586,-74.043117,3270,Jersey & 6th St,40.725289,-74.045572,24377,Subscriber,1989.0,1
1,606,2016-12-01 00:56:09,2016-12-01 01:06:16,3194,McGinley Square,40.725340,-74.067622,3189,West Side Light Rail,40.714402,-74.088772,26273,Subscriber,1962.0,1


In [13]:
df.tail(2)

,Trip Duration,Start Time,Stop Time,Start Station ID,Start Station Name,Start Station Latitude,Start Station Longitude,End Station ID,End Station Name,End Station Latitude,End Station Longitude,Bike ID,User Type,Birth Year,Gender
8248,943,2016-02-29 23:34:49,2016-02-29 23:50:33,3187,Warren St,40.721124,-74.038051,3202,Newport PATH,40.727224,-74.033759,24462,Subscriber,1980.0,1
8249,1955,2016-02-29 23:45:25,2016-03-01 00:18:01,3199,Newport Pkwy,40.728745,-74.032108,3199,Newport Pkwy,40.728745,-74.032108,24397,Subscriber,1975.0,1


In [7]:
duplicates = df.duplicated()
duplicates.value_counts()

False    247584
Name: count, dtype: int64

In [8]:
df.describe()

,Trip Duration,Start Station ID,Start Station Latitude,Start Station Longitude,End Station ID,End Station Latitude,End Station Longitude,Bike ID,Birth Year,Gender
count,2.475840e+05,247584.000000,247584.000000,247584.000000,247584.000000,247584.000000,247584.000000,247584.000000,228585.000000,247584.000000
mean,8.856305e+02,3207.065206,40.723121,-74.046438,3203.572553,40.722594,-74.045855,24935.260481,1979.335276,1.123534
std,3.593798e+04,26.955103,0.008199,0.011211,61.579494,0.007958,0.011283,748.469712,9.596809,0.518687
min,6.100000e+01,3183.000000,40.692640,-74.096937,147.000000,40.692216,-74.096937,14552.000000,1900.000000,0.000000
25%,2.480000e+02,3186.000000,40.717732,-74.050656,3186.000000,40.716540,-74.050444,24491.000000,1974.000000,1.000000
50%,3.900000e+02,3201.000000,40.721525,-74.044247,3199.000000,40.721124,-74.043117,24609.000000,1981.000000,1.000000
75%,6.660000e+02,3211.000000,40.727596,-74.038051,3211.000000,40.727224,-74.036486,24719.000000,1986.000000,1.000000
max,1.632981e+07,3426.000000,40.752559,-74.032108,3426.000000,40.801343,-73.957390,27274.000000,2000.000000,2.000000


In [11]:
(1-(df['Birth Year'].count()/df['Bike ID'].count()))*100

7.673759208995734